In [ ]:
# Importing Necessary Libraries
import os
import nltk
import math
import random
from collections import Counter
from collections import defaultdict


In [ ]:
# Tokenizer to keep only Alpha-Numeric Terms  [a-zA-Z0-9_]
tokenizer = nltk.RegexpTokenizer(r"\w+")

global corpus
corpus = {}

global index
index = defaultdict(list)

def PreProcess(data):
    data = ' '.join(tokenizer.tokenize(data))
    data = data.lower()
    return data

In [ ]:
# Function to Calculate Freequency of All Terms in Entire Corpus
def term_freq_corpus(corpus):
    term_freq_c = Counter({})
    for doc in corpus:
        term_freq_c += Counter(corpus[doc])
    return term_freq_c

In [ ]:
# Function to Calculate TF,IDF Values of a Term
def tfidf_val_calc(term,term_indxs_lst,ln_corpus,term_freq_c):
    # tf : Term Freequency of Term in Document, rounded off to 8 decimal places
    try:
        tf = round(((1 + math.log10(len(term_indxs_lst[term])))),8)
    except:
        tf = 0
    # idf : Inverse Document Freequency of Term in Corpus, rounded off to 8 decimal places
    try:
        idf = round((math.log10(ln_corpus / term_freq_c[term])),8)
    except:
        idf = 0
    return tf,idf

In [ ]:
# Function to Calculate Query Vector
def Query_Vector_Generator(query,index):
    query = PreProcess(query)
    query = query.split()
    query_vector = defaultdict(lambda: 0)
    for term in query:
        try:
            query_vector[term] = index[term][0]
        except:
            query_vector[term] = 0
    return query_vector

In [ ]:
def Create_IndexFn(stopwords_path,stopwords_filename,collection_path):
    # Getting StopWords from a File (Optional)
    '''
    For Windows Users, Use the following line of code to open file  
        with open(path+'\'+filename) as f:
    '''
    with open(stopwords_path+'/'+stopwords_filename) as f:
        global stopwords
        stopwords = f.read().split('\n')
        
    # Changing Working Directory to the Folder that contains Document Collection 
    os.chdir(collection_path)

    '''
    Reading, Pre-Processing Document Collection and Creating Corpus
    It is assumed that Document Collection is in '.txt' files

    For Windows Users, Use the following line of code to open file  
        file_path = f".\{file}"
    '''
    for file in os.listdir():
        if file.endswith(".txt"):
            file_path = f"./{file}"
            with open(file_path, "r") as f:
                doc_data = f.read()
            doc_data = PreProcess(doc_data)
            corpus[file[:-4]] = doc_data.split() # [:-4] to Remove '.txt' from FileName which will be used as DocId
    
    global ln_corpus 
    ln_corpus = len(corpus)
    
    term_freq_c = term_freq_corpus(corpus)
        # Create Required Index
    for docId in corpus:
        # term_indxs_lst Stores All Indexes Of a Term in a Document
        term_indxs_lst = defaultdict(list) 
        doc_data = corpus[docId]
        # Such Method Avoids Indexing of Same Term for the Same Document
        for i,term in enumerate(doc_data):
            if term not in stopwords:
                # By Convention, Indexing of Terms in a Document Starts at 1
                term_indxs_lst[term].append(i+1)
        for term in term_indxs_lst:
            # Calculate TF,IDF Values for the Term
            tf , idf = tfidf_val_calc(term,term_indxs_lst,ln_corpus,term_freq_c)
            # Weight = TF-IDF Value of the Term for the Particular Document
            wt = round((tf*idf),8)
            if len(index[term]) == 0:
                index[term].append(idf)
            index[term].append([docId, wt, term_indxs_lst[term]])
    # Return the Index
    return index